In [13]:
import torch
import torchvision
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import os
import os.path as osp
import numpy as np
from PIL import Image
import glob

import scipy.io as io
import numpy as np
from fire import Fire
from sklearn.linear_model import Perceptron
from sklearn.multiclass import OutputCodeClassifier

import pandas as pd
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


raw_table = pd.read_csv('task2/TrainSamples-300.csv', header=None)
train_data = np.array(raw_table).astype(np.float32)
print(train_data.shape)
# print(train_data[:10])

raw_table = pd.read_csv('task2/TrainLabels-300.csv', header=None)
train_label = np.array(raw_table).reshape(-1).astype(np.long)
print(train_label.shape)


(30000, 1024)
(30000,)


In [14]:


split_sep = 29000
test_data = train_data[split_sep:]
test_label = train_label[split_sep:]

train_data = train_data[:split_sep]
train_label = train_label[:split_sep]



class my_clsifer(nn.Module):
    def __init__(
        self,
        input_c,
        output_c,
    ):
        super(my_clsifer, self).__init__()
        self.func = nn.Sequential(
            nn.Linear(input_c, input_c * 2),
            nn.BatchNorm1d(input_c * 2),
            nn.ReLU(),
            nn.Linear(input_c * 2, input_c * 2),
            nn.BatchNorm1d(input_c * 2),
            nn.ReLU(),
            nn.Linear(input_c * 2, output_c * 10),
            nn.BatchNorm1d(output_c * 10),
            nn.ReLU(),
            nn.Linear(output_c * 10, output_c),
        )

        # self.a = nn.Linear(input_c, input_c * 2)
        # self.b = nn.BatchNorm1d(input_c * 2)
        # # nn.ReLU(),
        # self.c = nn.Linear(input_c * 2, input_c * 2)

    def forward(self, input):
        x = self.func(input)
        # x = self.a(input)
        # x = self.b(x)
        # x = self.c(x)
        return x


class my_dataset(data.Dataset):
    def __init__(self, datasource, labels, norm_bound=None):
        self.data_ = datasource
        self.labels = labels

        if norm_bound:
            self.data_ /= (norm_bound * 2)

    def __len__(self):
        return self.data_.shape[0]

    def __getitem__(self, index):
        return self.data_[index], self.labels[index]


my_model = my_clsifer(1024, 100).cuda()

dataloader = data.DataLoader(my_dataset(train_data, train_label),
                             batch_size=100,
                             shuffle=True,
                             pin_memory=True,
                             num_workers=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_model.parameters(), lr=5e-5)

test_loader = data.DataLoader(my_dataset(test_data, test_label),
                              batch_size=1,
                              shuffle=False,
                              pin_memory=True)


def test_(model, test_loader):
    model.eval()
    count = 0
    acc = 0
    with torch.no_grad():
        for iteration, batch in enumerate(test_loader):
            count += 1
            data_, label_ = batch
            data_, label_ = data_.cuda(), label_.cuda()
            output = model(data_.to(torch.float32))

            output = torch.argmax(output, dim=1)
            if int(output.item()) == label_.item():
                acc += 1

    return acc / count


epochs = 25
for i in range(epochs):
    for iteration, batch in enumerate(dataloader):
        # print(batch)
        # break

        my_model.train()
        data_, label_ = batch
        data_, label_ = data_.cuda(), label_.cuda()
        output = my_model(data_.to(torch.float32))

        loss_ = criterion(output, label_.long())
        loss_.backward()

        optimizer.step()
        optimizer.zero_grad()
        
        # if iteration % 100 == 0:

        #     acc = test_(my_model, test_loader)
        #     # acc = 0
        #     print('E: {}\t I: {}\t Loss:{:.6f}\t acc:{}\t'.format(i, iteration, loss_, acc))

    acc = test_(my_model, test_loader)
    # acc = 0
    print('E: {}\t I: {}\t  acc:{}\t'.format(i, iteration, acc))

torch.save(my_model, 'task2.pth')

E: 0	 I: 289	  acc:0.783	
E: 1	 I: 289	  acc:0.821	
E: 2	 I: 289	  acc:0.839	
E: 3	 I: 289	  acc:0.842	
E: 4	 I: 289	  acc:0.844	
E: 5	 I: 289	  acc:0.849	
E: 6	 I: 289	  acc:0.856	
E: 7	 I: 289	  acc:0.855	
E: 8	 I: 289	  acc:0.858	
E: 9	 I: 289	  acc:0.853	
E: 10	 I: 289	  acc:0.853	
E: 11	 I: 289	  acc:0.856	
E: 12	 I: 289	  acc:0.851	
E: 13	 I: 289	  acc:0.852	
E: 14	 I: 289	  acc:0.858	
E: 15	 I: 289	  acc:0.86	
E: 16	 I: 289	  acc:0.859	
E: 17	 I: 289	  acc:0.854	
E: 18	 I: 289	  acc:0.854	
E: 19	 I: 289	  acc:0.857	
E: 20	 I: 289	  acc:0.857	
E: 21	 I: 289	  acc:0.854	
E: 22	 I: 289	  acc:0.855	
E: 23	 I: 289	  acc:0.854	
E: 24	 I: 289	  acc:0.86	


In [15]:
raw_table = pd.read_csv('task2/TestSamples-300.csv', header=None)
test_data_new = np.array(raw_table).astype(np.float32)
print(test_data_new.shape)



class my_dataset(data.Dataset):
    def __init__(self, datasource, labels, norm_bound=None):
        self.data_ = datasource
        self.labels = labels

        if norm_bound:
            self.data_ /= (norm_bound * 2)

    def __len__(self):
        return self.data_.shape[0]

    def __getitem__(self, index):
        if self.labels is not None:
            return self.data_[index], self.labels[index]
        else:
            return self.data_[index]

test_loader = data.DataLoader(my_dataset(test_data_new, None),
                              batch_size=1,
                              shuffle=False,
                              pin_memory=True)

my_model.eval()

result = []
with torch.no_grad():
    for iteration, batch in enumerate(test_loader):
        data_ = batch
        data_ = data_.cuda()
        output = my_model(data_.to(torch.float32))

        output = torch.argmax(output, dim=1)
        result.append(output.item())

result = np.array(result, dtype=np.int).reshape(-1, 1)
data_frame = pd.DataFrame(result)

data_frame.to_csv('task2/TestLabels__21S003010.csv', header=0, index=0)

(10000, 1024)
